## sentdex - 딥러닝 Pt.8

<h3> 목표 : RNN으로 암호화폐 가격 예측하기</h3>
<hr>

**&lt;데이터 다운로드&gt;**   
https://pythonprogramming.net/static/downloads/machine-learning-data/crypto_data.zip

In [2]:
# 60분을 통해 3분 (미래) 예측

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = "LTC-USD"

**1 : 매수, 0 : 매도 (신호)**

In [3]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

**target class를 만들기 위해서, " f"{ratio}_close " 열의 3일 뒤 가격에 해당하는 "future" 열을 만든다.**

In [4]:
import pandas as pd
import os

main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    
    df = pd.read_csv(dataset,
                     names = ["time", "low", "high", "open", "close", "volume"])
    df.rename(columns = {"close" : f"{ratio}_close", "volume" : f"{ratio}_volume"}, inplace = True)
    
    df.set_index("time", inplace = True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

**이제 <u> main_df </u>는 각 암호화폐에 대한 "close" & "volume" 데이터를 포함한다.** <br>
**위에서 정의한 함수를 사용해서 target column을 만들어보자!**

In [5]:
# "future" value gonna be 3-days-shifted value of "close" coluomn
main_df["future"] = main_df[ f"{RATIO_TO_PREDICT}_close" ].shift(-FUTURE_PERIOD_PREDICT)        

# Classify! (Not ML tho)
main_df['target'] = list(map(classify, main_df[ f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

print(main_df[ [ f"{RATIO_TO_PREDICT}_close", "future", "target"] ].head(10))

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0
